# Chapter 05-02: CNN 아키텍처 — LeNet부터 EfficientNet까지

## 학습 목표
- 주요 CNN 아키텍처의 역사적 발전 흐름을 이해한다
- LeNet-5, VGG 블록 패턴을 직접 구현할 수 있다
- ResNet의 잔차 연결(Skip Connection) 원리를 이해하고 구현한다
- `tf.keras.applications`로 사전 학습 모델을 불러와 활용할 수 있다

## 목차
1. [주요 CNN 아키텍처 역사](#1)
2. [LeNet-5 구현](#2)
3. [VGG 블록 패턴](#3)
4. [ResNet 잔차 블록](#4)
5. [사전 학습 모델 (tf.keras.applications)](#5)
6. [정리](#6)

In [ ]:
# 필수 라이브러리 임포트
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

# 한글 폰트 설정
plt.rcParams['font.family'] = 'AppleGothic'
plt.rcParams['axes.unicode_minus'] = False

# 재현성 시드 설정
tf.random.set_seed(42)
np.random.seed(42)

print(f'TensorFlow 버전: {tf.__version__}')

## 1. 주요 CNN 아키텍처 역사 <a id='1'></a>

CNN 아키텍처는 지난 30여 년간 급격한 발전을 이루었다.

| 연도 | 모델 | 주요 특징 | ImageNet Top-5 오류율 |
|------|------|-----------|----------------------|
| 1998 | **LeNet-5** | 최초의 실용적 CNN, 손글씨 인식 | - |
| 2012 | **AlexNet** | GPU 학습, ReLU, Dropout 적용 | 15.3% |
| 2014 | **VGGNet** | 3×3 Conv 반복, 깊이 강조 | 7.3% |
| 2015 | **ResNet** | 잔차 연결(Skip Connection), 152층 | 3.6% |
| 2017 | **MobileNet** | Depthwise Separable Conv, 경량화 | - |
| 2019 | **EfficientNet** | 복합 스케일링(Compound Scaling) | 2.9% |

### 핵심 발전 방향
- **더 깊게**: LeNet(5층) → ResNet(152층)
- **더 효율적으로**: VGG → MobileNet → EfficientNet
- **경사 소실 해결**: ResNet의 Skip Connection이 핵심 돌파구

### 아키텍처 발전 타임라인
```
1998       2012        2014      2015      2017        2019
LeNet-5 → AlexNet → VGGNet → ResNet → MobileNet → EfficientNet
  5층      8층        19층      152층      28층          ---
```

## 2. LeNet-5 구현 <a id='2'></a>

LeNet-5 (LeCun et al., 1998)는 우편번호 인식을 위해 설계된 최초의 실용적 CNN이다.

구조:
```
입력(32×32) → Conv(6) → AvgPool → Conv(16) → AvgPool → Flatten → FC(120) → FC(84) → FC(10)
```

원본은 Tanh 활성화를 사용했으나, 현대적 버전에서는 ReLU를 사용한다.

In [ ]:
# LeNet-5 구현 (현대적 버전: ReLU 활성화)
def build_lenet5(input_shape=(32, 32, 1), num_classes=10):
    """
    LeNet-5 아키텍처 구현
    
    Args:
        input_shape: 입력 이미지 형태 (H, W, C)
        num_classes: 분류 클래스 수
    
    Returns:
        tf.keras.Sequential 모델
    """
    model = tf.keras.Sequential([
        # C1: 첫 번째 합성곱 레이어 (5x5 커널, 6 필터)
        tf.keras.layers.Conv2D(6, kernel_size=5, activation='relu',
                               input_shape=input_shape, name='C1_conv'),
        
        # S2: 평균 풀링 (원본 LeNet은 AvgPooling 사용)
        tf.keras.layers.AveragePooling2D(pool_size=2, strides=2, name='S2_pool'),
        
        # C3: 두 번째 합성곱 레이어 (5x5 커널, 16 필터)
        tf.keras.layers.Conv2D(16, kernel_size=5, activation='relu', name='C3_conv'),
        
        # S4: 평균 풀링
        tf.keras.layers.AveragePooling2D(pool_size=2, strides=2, name='S4_pool'),
        
        # C5: 세 번째 합성곱 (1x1 출력 → Fully Connected와 동일)
        tf.keras.layers.Conv2D(120, kernel_size=5, activation='relu', name='C5_conv'),
        
        # 평탄화
        tf.keras.layers.Flatten(name='flatten'),
        
        # F6: 완전 연결층
        tf.keras.layers.Dense(84, activation='relu', name='F6_dense'),
        
        # 출력층
        tf.keras.layers.Dense(num_classes, activation='softmax', name='output')
    ], name='LeNet-5')
    
    return model


# LeNet-5 모델 생성 및 구조 출력
lenet = build_lenet5(input_shape=(32, 32, 1), num_classes=10)
lenet.summary()

# MNIST 데이터로 빠른 테스트
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.mnist.load_data()

# 28x28 → 32x32 리사이즈 (LeNet-5 입력 크기)
x_train_resized = tf.image.resize(x_train[..., np.newaxis], [32, 32]) / 255.0
x_test_resized  = tf.image.resize(x_test[..., np.newaxis],  [32, 32]) / 255.0

lenet.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# 빠른 학습 (3 에폭만)
history_lenet = lenet.fit(
    x_train_resized, y_train,
    epochs=3,
    batch_size=256,
    validation_split=0.1,
    verbose=1
)

test_loss, test_acc = lenet.evaluate(x_test_resized, y_test, verbose=0)
print(f'\nLeNet-5 테스트 정확도: {test_acc:.4f}')

## 3. VGG 블록 패턴 <a id='3'></a>

VGGNet (Simonyan & Zisserman, 2014)의 핵심 아이디어:
- **3×3 Conv만 사용**: 작은 커널을 여러 번 쌓으면 큰 수용 영역(Receptive Field)을 얻을 수 있다
  - 3×3 Conv 2개 = 5×5 Conv 1개와 동일한 수용 영역, 하지만 파라미터 수는 더 적다
  - $2 \times (3^2 \times C^2) = 18C^2$ vs $5^2 \times C^2 = 25C^2$
- **일정한 VGG 블록**: Conv → Conv → MaxPool 패턴 반복
- **채널 수 점진적 증가**: 64 → 128 → 256 → 512

In [ ]:
def vgg_block(num_convs, num_filters):
    """
    VGG 블록 생성 함수
    
    Args:
        num_convs: 블록 내 합성곱 레이어 수
        num_filters: 필터 수 (출력 채널 수)
    
    Returns:
        tf.keras.Sequential 블록
    """
    block = tf.keras.Sequential(name=f'vgg_block_{num_filters}ch')
    
    # num_convs개의 3x3 Conv 레이어 반복
    for _ in range(num_convs):
        block.add(tf.keras.layers.Conv2D(
            num_filters, kernel_size=3, padding='same', activation='relu'
        ))
    
    # 블록 끝에 MaxPooling으로 공간 크기 절반 축소
    block.add(tf.keras.layers.MaxPooling2D(pool_size=2, strides=2))
    
    return block


def build_mini_vgg(input_shape=(32, 32, 3), num_classes=10):
    """
    CIFAR-10용 미니 VGG 모델
    (원본 VGG-16은 224x224 입력 → 32x32에 맞게 축소)
    """
    inputs = tf.keras.Input(shape=input_shape)
    
    # VGG 블록 스택 (필터 수 점진적 증가)
    x = vgg_block(num_convs=2, num_filters=64)(inputs)   # 32→16
    x = vgg_block(num_convs=2, num_filters=128)(x)        # 16→8
    x = vgg_block(num_convs=2, num_filters=256)(x)        # 8→4
    
    # 분류 헤드
    x = tf.keras.layers.Flatten()(x)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.Dense(512, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.5)(x)
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    
    return tf.keras.Model(inputs, outputs, name='Mini-VGG')


# Mini-VGG 모델 생성 및 구조 확인
mini_vgg = build_mini_vgg(input_shape=(32, 32, 3), num_classes=10)
mini_vgg.summary()

# 파라미터 수 비교
print(f'\nMini-VGG 총 파라미터: {mini_vgg.count_params():,}')
print(f'LeNet-5 총 파라미터:  {lenet.count_params():,}')
print(f'파라미터 차이: {mini_vgg.count_params() / lenet.count_params():.1f}배')

## 4. ResNet 잔차 블록 <a id='4'></a>

### 경사 소실 문제 (Vanishing Gradient Problem)

네트워크가 깊어질수록 역전파 시 경사가 점점 작아져 초기 레이어가 학습되지 않는 문제가 발생한다.

### ResNet의 핵심 아이디어: Skip Connection

He et al. (2015)는 **잔차 연결(Residual Connection)**로 이 문제를 해결했다:

$h_l = F(x_l, W_l) + x_l$

여기서:
- $x_l$: 레이어 $l$의 입력
- $F(x_l, W_l)$: Conv → BN → ReLU → Conv → BN 블록의 출력 (잔차, Residual)
- $h_l$: 최종 출력 (잔차 + 항등 입력)

이 구조는 경사를 최소 $1$로 유지하여 매우 깊은 네트워크(100층 이상)도 학습 가능하게 한다:

$\frac{\partial h_l}{\partial x_l} = \frac{\partial F}{\partial x_l} + 1$

즉, 항등(identity) 경로가 항상 경사를 보존한다.

In [ ]:
def residual_block(x, filters, kernel_size=3, stride=1, use_projection=False):
    """
    ResNet 잔차 블록 구현 (Functional API)
    
    구조: Conv → BN → ReLU → Conv → BN → (+ shortcut) → ReLU
    
    Args:
        x: 입력 텐서
        filters: 출력 채널 수
        kernel_size: 합성곱 커널 크기
        stride: 첫 번째 Conv의 스트라이드 (2이면 다운샘플링)
        use_projection: 입출력 채널이 다를 때 shortcut에 1x1 Conv 적용 여부
    
    Returns:
        잔차 블록 출력 텐서
    """
    shortcut = x  # 항등 경로 저장
    
    # 주 경로: F(x, W)
    # --- 첫 번째 Conv ---
    x = tf.keras.layers.Conv2D(
        filters, kernel_size, strides=stride, padding='same', use_bias=False
    )(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    # --- 두 번째 Conv ---
    x = tf.keras.layers.Conv2D(
        filters, kernel_size, strides=1, padding='same', use_bias=False
    )(x)
    x = tf.keras.layers.BatchNormalization()(x)
    
    # 차원이 맞지 않을 때 1x1 Conv로 프로젝션 (Projection Shortcut)
    if use_projection or stride != 1 or shortcut.shape[-1] != filters:
        shortcut = tf.keras.layers.Conv2D(
            filters, kernel_size=1, strides=stride, padding='same', use_bias=False
        )(shortcut)
        shortcut = tf.keras.layers.BatchNormalization()(shortcut)
    
    # Skip Connection: F(x) + x
    x = tf.keras.layers.Add()([x, shortcut])
    x = tf.keras.layers.Activation('relu')(x)
    
    return x


def build_mini_resnet(input_shape=(32, 32, 3), num_classes=10):
    """CIFAR-10용 미니 ResNet 구현"""
    inputs = tf.keras.Input(shape=input_shape)
    
    # 초기 합성곱
    x = tf.keras.layers.Conv2D(64, 3, padding='same', use_bias=False)(inputs)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Activation('relu')(x)
    
    # 잔차 블록 스택
    x = residual_block(x, filters=64)                            # 32x32
    x = residual_block(x, filters=64)
    
    x = residual_block(x, filters=128, stride=2)                 # 16x16 (다운샘플링)
    x = residual_block(x, filters=128)
    
    x = residual_block(x, filters=256, stride=2)                 # 8x8 (다운샘플링)
    x = residual_block(x, filters=256)
    
    # 분류 헤드
    x = tf.keras.layers.GlobalAveragePooling2D()(x)              # Flatten 대신 GAP 사용
    outputs = tf.keras.layers.Dense(num_classes, activation='softmax')(x)
    
    return tf.keras.Model(inputs, outputs, name='Mini-ResNet')


# Mini-ResNet 생성 및 구조 확인
mini_resnet = build_mini_resnet(input_shape=(32, 32, 3), num_classes=10)
mini_resnet.summary()

print(f'\nMini-ResNet 총 파라미터: {mini_resnet.count_params():,}')

## 5. 사전 학습 모델 (tf.keras.applications) <a id='5'></a>

`tf.keras.applications`는 ImageNet으로 사전 학습된 다양한 모델을 제공한다.

### 주요 모델 비교

| 모델 | 파라미터 수 | Top-1 정확도 | 특징 |
|------|------------|-------------|------|
| ResNet50 | 25.6M | 74.9% | 잔차 연결, 균형적 성능 |
| MobileNetV2 | 3.5M | 71.3% | 경량화, 모바일 친화적 |
| EfficientNetB0 | 5.3M | 77.1% | 복합 스케일링 (레거시 — V2로 대체 권장) || EfficientNetV2B0 | 7.1M | 78.7% | 개선된 정규화·훈련 효율, 권장 최신 버전 (2026-02-25 기준) |\n

In [ ]:
# tf.keras.applications에서 사전 학습 모델 로드
# include_top=False: ImageNet 분류 헤드 제거 (전이학습용)
# weights='imagenet': ImageNet 사전 학습 가중치 사용

models_to_compare = [
    {
        'name': 'ResNet50',
        'loader': tf.keras.applications.ResNet50,
        'input_size': 224
    },
    {
        'name': 'MobileNetV2',
        'loader': tf.keras.applications.MobileNetV2,
        'input_size': 224
    },
    {
        'name': 'EfficientNetV2B0',
        'loader': tf.keras.applications.EfficientNetV2B0,
        'input_size': 224
    }
]

print(f'{'모델명':<20} {'총 파라미터':>15} {'학습 가능 파라미터':>20} {'입력 크기':>10}')
print('=' * 70)

loaded_models = {}
for cfg in models_to_compare:
    # 모델 로드 (include_top=False: 분류 헤드 제외)
    base_model = cfg['loader'](
        include_top=False,
        weights='imagenet',
        input_shape=(cfg['input_size'], cfg['input_size'], 3)
    )
    
    total_params     = base_model.count_params()
    trainable_params = sum([tf.size(w).numpy() for w in base_model.trainable_weights])
    
    print(f"{cfg['name']:<20} {total_params:>15,} {trainable_params:>20,} "
          f"{cfg['input_size']}x{cfg['input_size']:>3}")
    
    loaded_models[cfg['name']] = base_model

print('\n출력 형태 확인:')
for name, model in loaded_models.items():
    dummy = tf.random.normal([1, 224, 224, 3])
    out = model(dummy, training=False)
    print(f'  {name}: 출력 형태 = {out.shape}')

In [ ]:
# 모델 아키텍처 시각화: 레이어 수와 파라미터 수 비교
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

model_names = ['LeNet-5', 'Mini-VGG', 'Mini-ResNet', 'ResNet50', 'MobileNetV2', 'EfficientNetV2B0']
param_counts = [
    lenet.count_params(),
    mini_vgg.count_params(),
    mini_resnet.count_params(),
    loaded_models['ResNet50'].count_params(),
    loaded_models['MobileNetV2'].count_params(),
    loaded_models['EfficientNetV2B0'].count_params()
]
layer_counts = [
    len(lenet.layers),
    len(mini_vgg.layers),
    len(mini_resnet.layers),
    len(loaded_models['ResNet50'].layers),
    len(loaded_models['MobileNetV2'].layers),
    len(loaded_models['EfficientNetV2B0'].layers)
]

colors = ['#FF6B6B', '#FFA500', '#4ECDC4', '#45B7D1', '#96CEB4', '#88D8B0']

# 파라미터 수 막대 그래프
bars1 = axes[0].bar(model_names, [p/1e6 for p in param_counts], color=colors)
axes[0].set_xlabel('모델')
axes[0].set_ylabel('파라미터 수 (백만)')
axes[0].set_title('모델별 파라미터 수 비교')
axes[0].tick_params(axis='x', rotation=30)
for bar, count in zip(bars1, param_counts):
    axes[0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.1,
                 f'{count/1e6:.1f}M', ha='center', va='bottom', fontsize=9)

# 레이어 수 막대 그래프
bars2 = axes[1].bar(model_names, layer_counts, color=colors)
axes[1].set_xlabel('모델')
axes[1].set_ylabel('레이어 수')
axes[1].set_title('모델별 레이어 수 비교')
axes[1].tick_params(axis='x', rotation=30)
for bar, count in zip(bars2, layer_counts):
    axes[1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.5,
                 str(count), ha='center', va='bottom', fontsize=9)

plt.suptitle('CNN 아키텍처 비교', fontsize=14)
plt.tight_layout()
plt.show()

## 6. 정리 <a id='6'></a>

### 아키텍처별 특징 비교 표

| 항목 | LeNet-5 | VGGNet | ResNet | MobileNet | EfficientNet |
|------|---------|--------|--------|-----------|-------------|
| 연도 | 1998 | 2014 | 2015 | 2017 | 2019 |
| 커널 크기 | 5×5 | 3×3 | 3×3 | Depthwise | 3×3 |
| 핵심 아이디어 | 최초 CNN | 깊이 증가 | Skip Connection | 경량화 | 복합 스케일링 |
| 활성화 함수 | Tanh | ReLU | ReLU | ReLU6 | Swish |
| 정규화 | 없음 | Dropout | BatchNorm | BatchNorm | BatchNorm |
| 모바일 적합성 | 낮음 | 낮음 | 중간 | 높음 | 높음 |

### 핵심 교훈
1. **깊이의 한계**: 단순히 층을 쌓으면 경사 소실 문제 발생 → ResNet의 Skip Connection으로 해결
2. **효율성 추구**: 정확도뿐 아니라 파라미터 효율도 중요 → MobileNet, EfficientNet
3. **전이학습 활용**: 처음부터 학습보다 사전 학습 모델을 활용하는 것이 현실적

### 다음 챕터 예고
**Chapter 05-03**: 사전 학습 모델을 이용한 전이학습 (Transfer Learning)을 실습한다.
Feature Extraction과 Fine-Tuning 전략을 단계별로 구현한다.